In [57]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import joblib
import sklearn
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [58]:
df = pd.read_csv('../dataset/hao/hao_dataset.csv')
df

,doi,in_source,in_amount_mmol,p_source,p_amount_mmol,sol,sol_amount_ml,TOP,TOP_amount_mmol,acid,...,other,other_amount_mmol,total_volume_ml,temp_c,time_min,diameter_nm,abs_nm,emission_nm,date_input,user
0,10.1080/15980316.2012.683537,indium myristate,0.10,tris(trimethylsilyl)phosphine,0.10,octadecene,8.0,None,0.000000,None,...,None,0.000000,8.012,300,30.0,2.3,436,481,6/25/2021,Hao
1,10.1016/j.matchemphys.2005.07.041,indium chloride,10.00,sodium phosphide,10.00,4-ethylpyridine,50.0,trioctylphosphine,44.841900,None,...,None,0.000000,70.000,130,2880.0,5,587,None,6/25/2021,Hao
2,10.1016/S0167-577X(03)00371-9,indium chloride,10.00,sodium phosphide,11.30,dimethylformamide,90.0,None,0.000000,None,...,None,0.000000,90.000,160,120.0,None,465,550,6/25/2021,Hao
3,10.1021/acs.chemmater.7b04069,"indium tris(N,N-diisopropylacetamidinato)",0.90,tris(trimethylsilyl)phosphine,0.15,mesitylene,5.0,None,0.000000,palmitic acid,...,None,0.000000,5.077,150,1260.0,2,523,None,6/26/2021,Hao
4,10.1021/acs.chemmater.7b04069,"indium tris(N,N-diisopropylacetamidinato)",0.90,tris(trimethylsilyl)phosphine,0.15,mesitylene,5.0,None,0.000000,palmitic acid,...,None,0.000000,5.077,230,1260.0,4.1,584,None,6/26/2021,Hao
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,10.1021/jp9704731,indium oxalate,2.33,tris(trimethylsilyl)phosphine,3.00,None,0.0,trioctylphosphine,24.552424,None,...,trioctylphosphine oxide,1.011111,11.962,300,4320.0,6,729,775,6/25/2021,Hao
242,10.1016/j.jcis.2010.06.037,chloroindium oxalate,4.20,tris(trimethylsilyl)phosphine,2.07,None,0.0,trioctylphosphine,13.452570,None,...,trioctylphosphine oxide,11.111111,17.112,280,4320.0,None,599,612,6/25/2021,Hao
243,Nayon,indium chloride,2.26,tris(trimethylsilyl)phosphine,2.00,None,0.0,trioctylphosphine,17.936760,None,...,trioctylphosphine oxide,0.830000,8.830,270,1140.0,5,633,653,7/11/2021,Nayon
244,10.1002/adom.201901362,indium chloride,0.10,tris(trimethylsilyl)phosphine,0.10,None,0.0,trioctylphosphine,11.210475,None,...,None,0.000000,5.000,280,60.0,2.3,423,485,7/23/2021,Hao


In [59]:
#Checks if there are any columns with no values
df.isna().sum()

doi                  0
in_source            0
in_amount_mmol       0
p_source             0
p_amount_mmol        0
sol                  0
sol_amount_ml        0
TOP                  0
TOP_amount_mmol      0
acid                 0
acid_amount_mmol     0
amine                0
amine_amount_mmol    0
thiol                0
thiol_amount_mmol    0
zinc                 0
zinc_amount_mmol     0
other                0
other_amount_mmol    0
total_volume_ml      0
temp_c               0
time_min             0
diameter_nm          0
abs_nm               0
emission_nm          0
date_input           0
user                 0
dtype: int64

In [60]:
#Separate out initial DataFrame into the input features and output features
df_input = df.drop(columns =['diameter_nm', 'abs_nm', 'emission_nm','doi','user','date_input'], inplace = False, axis = 1)
df_output = df[['diameter_nm', 'abs_nm', 'emission_nm']]

In [61]:
#Checks the column names, and ensures that they do not have any leading or trailing spaces
df_input.columns = df_input.columns.str.strip()
df_output.columns = df_output.columns.str.strip()

In [62]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   in_source          246 non-null    object 
 1   in_amount_mmol     246 non-null    float64
 2   p_source           246 non-null    object 
 3   p_amount_mmol      246 non-null    float64
 4   sol                246 non-null    object 
 5   sol_amount_ml      246 non-null    float64
 6   TOP                246 non-null    object 
 7   TOP_amount_mmol    246 non-null    float64
 8   acid               246 non-null    object 
 9   acid_amount_mmol   246 non-null    float64
 10  amine              246 non-null    object 
 11  amine_amount_mmol  246 non-null    float64
 12  thiol              246 non-null    object 
 13  thiol_amount_mmol  246 non-null    float64
 14  zinc               246 non-null    object 
 15  zinc_amount_mmol   246 non-null    float64
 16  other              246 non

In [63]:
df_input['temp_c'] = df_input['temp_c'].astype(float)
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   in_source          246 non-null    object 
 1   in_amount_mmol     246 non-null    float64
 2   p_source           246 non-null    object 
 3   p_amount_mmol      246 non-null    float64
 4   sol                246 non-null    object 
 5   sol_amount_ml      246 non-null    float64
 6   TOP                246 non-null    object 
 7   TOP_amount_mmol    246 non-null    float64
 8   acid               246 non-null    object 
 9   acid_amount_mmol   246 non-null    float64
 10  amine              246 non-null    object 
 11  amine_amount_mmol  246 non-null    float64
 12  thiol              246 non-null    object 
 13  thiol_amount_mmol  246 non-null    float64
 14  zinc               246 non-null    object 
 15  zinc_amount_mmol   246 non-null    float64
 16  other              246 non

In [64]:
#Initializes 2 lists to contain all of the numerical and categorical input columns
input_num_cols = [col for col in df_input.columns if df[col].dtypes !='O']
input_cat_cols = [col for col in df_input.columns if df[col].dtypes =='O']

In [65]:
input_cat_cols

['in_source',
 'p_source',
 'sol',
 'TOP',
 'acid',
 'amine',
 'thiol',
 'zinc',
 'other']

In [66]:
#Initializes the ColumnTransformer object, and specifies what it will do with a passed in dataframe
ct = ColumnTransformer([
    ('step1', StandardScaler(), input_num_cols),
    ('step2', OneHotEncoder(sparse=False, handle_unknown='ignore'), input_cat_cols)
], remainder = 'passthrough')

In [67]:
#Uses the ColumnTransformer object to modify the input columns
df_input_scaled_encoded = pd.DataFrame(ct.fit_transform(df_input))
df_input_scaled_encoded

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-0.587378,-0.622362,-0.198864,-0.286002,-0.420577,-0.596494,0.378054,-0.498460,-0.254494,-0.319687,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.946556,6.875236,2.139673,7.645704,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,3.128147,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7.946556,7.859769,4.366852,-0.286002,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,4.240567,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,-0.482935,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,-0.482935,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,1.334912,1.573904,-0.644300,4.056869,-0.420577,-0.596494,-0.185590,-0.581011,0.653295,-0.099984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
242,2.946878,0.869585,-0.644300,2.093509,-0.420577,-0.596494,-0.185590,-0.581011,9.721213,0.186464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
243,1.274571,0.816571,-0.644300,2.886680,-0.420577,-0.596494,-0.185590,-0.581011,0.490691,-0.274189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
244,-0.587378,-0.622362,-0.644300,1.696924,-0.420577,-0.596494,-0.185590,1.070009,-0.254494,-0.487218,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
ct.transformers_

[('step1',
  StandardScaler(),
  ['in_amount_mmol',
   'p_amount_mmol',
   'sol_amount_ml',
   'TOP_amount_mmol',
   'acid_amount_mmol',
   'amine_amount_mmol',
   'thiol_amount_mmol',
   'zinc_amount_mmol',
   'other_amount_mmol',
   'total_volume_ml',
   'temp_c',
   'time_min']),
 ('step2',
  OneHotEncoder(handle_unknown='ignore', sparse=False),
  ['in_source',
   'p_source',
   'sol',
   'TOP',
   'acid',
   'amine',
   'thiol',
   'zinc',
   'other'])]

In [69]:
#Look into ct object to see the column titles
array_of_cat_titles = ct.transformers_[1][1].get_feature_names()
len(array_of_cat_titles) #There are a total of 41 encoded categorical columns

62

In [70]:
#Number of numerical columns in the dataset
len(input_num_cols)

12

In [71]:
for i in range(len(input_num_cols)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i]: input_num_cols[i]}, inplace = True)

for j in range(len(array_of_cat_titles)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i+1]: array_of_cat_titles[j]}, inplace = True)
    i = i + 1

In [72]:
df_input_scaled_encoded

,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,total_volume_ml,...,x8_None,x8_acetic acid,x8_copper bromide,x8_dioctyl ether,x8_oleic acid,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide
0,-0.587378,-0.622362,-0.198864,-0.286002,-0.420577,-0.596494,0.378054,-0.498460,-0.254494,-0.319687,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7.946556,6.875236,2.139673,7.645704,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,3.128147,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7.946556,7.859769,4.366852,-0.286002,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,4.240567,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,-0.482935,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,-0.482935,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,1.334912,1.573904,-0.644300,4.056869,-0.420577,-0.596494,-0.185590,-0.581011,0.653295,-0.099984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
242,2.946878,0.869585,-0.644300,2.093509,-0.420577,-0.596494,-0.185590,-0.581011,9.721213,0.186464,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
243,1.274571,0.816571,-0.644300,2.886680,-0.420577,-0.596494,-0.185590,-0.581011,0.490691,-0.274189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
244,-0.587378,-0.622362,-0.644300,1.696924,-0.420577,-0.596494,-0.185590,1.070009,-0.254494,-0.487218,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
df_scaled_encoded = pd.concat([df_input_scaled_encoded, df_output], axis = 1)
df_scaled_encoded

,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,total_volume_ml,...,x8_dioctyl ether,x8_oleic acid,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm
0,-0.587378,-0.622362,-0.198864,-0.286002,-0.420577,-0.596494,0.378054,-0.498460,-0.254494,-0.319687,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,436,481
1,7.946556,6.875236,2.139673,7.645704,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,3.128147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,587,None
2,7.946556,7.859769,4.366852,-0.286002,-0.420577,-0.596494,-0.185590,-0.581011,-0.254494,4.240567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,465,550
3,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,-0.482935,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,523,None
4,0.102233,-0.584495,-0.365902,-0.286002,-0.305227,-0.552900,-0.185590,-0.581011,-0.254494,-0.482935,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,584,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,1.334912,1.573904,-0.644300,4.056869,-0.420577,-0.596494,-0.185590,-0.581011,0.653295,-0.099984,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,729,775
242,2.946878,0.869585,-0.644300,2.093509,-0.420577,-0.596494,-0.185590,-0.581011,9.721213,0.186464,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,None,599,612
243,1.274571,0.816571,-0.644300,2.886680,-0.420577,-0.596494,-0.185590,-0.581011,0.490691,-0.274189,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5,633,653
244,-0.587378,-0.622362,-0.644300,1.696924,-0.420577,-0.596494,-0.185590,1.070009,-0.254494,-0.487218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,423,485


In [76]:
df_scaled_encoded.to_csv('hao_dataset_scaled.csv')

In [78]:
#Saves the row indexes to drop for absorbance modelling into a list
total_row_num = len(df_scaled_encoded)
drop_list_abs =[]
for row_i in range(total_row_num):
    if df_scaled_encoded['abs_nm'].values[row_i] == 'None':
        drop_list_abs.append(row_i)
    
len(drop_list_abs)

14

In [79]:
#Drops the appropriate rows
df_absorbance_scaled_encoded = df_scaled_encoded.drop(drop_list_abs)
#Saves the data for absorbance modelling to CSV
df_absorbance_scaled_encoded.to_csv('hao_all_abs_scaled.csv')

In [80]:
#Saves the row indexes to drop for absorbance modelling into a list
total_row_num = len(df_scaled_encoded)
drop_list_emi =[]
for row_i in range(total_row_num):
    if df_scaled_encoded['emission_nm'].values[row_i] == 'None':
        drop_list_emi.append(row_i)
    
len(drop_list_emi)

136

In [81]:
#Drops the appropriate rows
df_emission_scaled_encoded = df_scaled_encoded.drop(drop_list_emi)
#Saves the data for absorbance modelling to CSV
df_emission_scaled_encoded.to_csv('hao_all_emi_scaled.csv')

In [82]:
#Saves the row indexes to drop for diameter modelling into a list
total_row_num = len(df_scaled_encoded)
drop_list_dia =[]
for row_i in range(total_row_num):
    if df_scaled_encoded['diameter_nm'].values[row_i] == 'None':
        drop_list_dia.append(row_i)
    
len(drop_list_dia)

175

In [84]:
#Drops the appropriate rows
df_dia_scaled_encoded = df_scaled_encoded.drop(drop_list_dia)
#Saves the data for absorbance modelling to CSV
df_dia_scaled_encoded.to_csv('hao_all_dia_scaled.csv')